**Olympics_study**



This project merely started with my excitement to watch every competition at Olympics and the love to play with data. Like many of us, knowing the process behind all the hardship of athletes made me curious to know how much past data is useful for future planning and analysis. I decided to do some number crunching on 124 years of Olympics to see which are the best performing countries at the Olympics and what makes them great!
I have gathered data of Olympics performances from year 1900 to 2020 and in their relation to the country’s GDP, population, infra-structure, economics.

I use the [olympics dataset](https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results),[2021 olympics report](https://www.kaggle.com/berkayalan/2021-olympics-medals-in-tokyo) from kaggle, and merged it with the country wise [gdp](https://www.kaggle.com/resulcaliskan/countries-gdps) and [population data](https://www.kaggle.com/centurion1986/countries-population).



In [ ]:
import os
import numpy as np 
import pandas as pd
#from matplotlib import pyplot as plt
#import seaborn as sns

The first five rows of the olympics data are shown below. We have 271,116 rows and 15 columns. Variables include Name, Sex, Age, Height, Weight of the athlete,their team name, sport, event and the year, season, city of the olympics he/she took part in. In addition, the data captures the medal won (if any) by the athlete.

In [ ]:
# Read in the data set
olympics = pd.read_csv('../input/120-years-of-olympic-history-athletes-and-results/athlete_events.csv')
olympics.head()

# Initial Data Cleaning and Exploration

**1) Missing Values** <br>
Finding and printing column wise missing values we have in the dataset 'Olympics'

In [ ]:
print(olympics.isnull().sum())

We can see Height,Weight and Age have a lot of missing values. Medals have a NaN in about 2,31,333 rows. This can be explained since not all participating athletes have won medals. Let's replace these missing values by 'Did not win' or 'DNW'.

In [ ]:
olympics['Medal'].fillna('DNW', inplace = True)

In [ ]:
# As expected the NaNs in the 'Medal' column disappear!
print(olympics.isnull().sum())

**2) NOC - National Olympic Committee** <br>
The organizations which send their sport persons to participate in the Olympics.
Are all NOCs linked to a unique team? We can find this out by taking a unique subset 
of just the NOC and team columns and taking a value count.

In [ ]:
print(olympics.loc[:, ['NOC', 'Team']].drop_duplicates()['NOC'].value_counts().head())

So NOC code 'FRA' is associated with 160 teams? That sounds prepostorous! Let's use a master of NOC to country mapping to correct this.

The NOC dataset has the NOC code and the corresponding Country Name. The first five rows of the data are shown below -

In [ ]:
# Lets read in the noc_country mapping first
noc_country = pd.read_csv('../input/120-years-of-olympic-history-athletes-and-results/noc_regions.csv')
noc_country.drop('notes', axis = 1 , inplace = True)
noc_country.rename(columns = {'region':'Country'}, inplace = True)

noc_country.head()

We will merge the original dataset with the NOC master using the **NOC code as the primary key**. This has to be a left join since we want all participating countries to remain in the data even if their NOC-Country is not found in the master. We can easily correct those manually.

In [ ]:
# merging
olympics_merge = olympics.merge(noc_country,
                                left_on = 'NOC',
                                right_on = 'NOC',
                                how = 'left')

Even after merging, we can see that the below NOC codes in the Olympics data had no counterpart in the NOC master data.

In [ ]:
# Do we have NOCs that didnt have a matching country in the master?
olympics_merge.loc[olympics_merge['Country'].isnull(),['NOC', 'Team']].drop_duplicates()

Looking at their 'Team' names we can manually insert the correct values into the olympics data.

Let's put these values in Country, which we can confirm from noc_country - <br>
    1. SGP - Singapore
    2. ROT - Refugee Olympic Athletes
    3. UNK - Unknown
    4. TUV - Tuvalu 

In [ ]:
# Replace missing Teams by the values above.

olympics_merge['Country'] = np.where(olympics_merge['NOC']=='SGP', 'Singapore', olympics_merge['Country'])
olympics_merge['Country'] = np.where(olympics_merge['NOC']=='ROT', 'Refugee Olympic Athletes', olympics_merge['Country'])
olympics_merge['Country'] = np.where(olympics_merge['NOC']=='UNK', 'Unknown', olympics_merge['Country'])
olympics_merge['Country'] = np.where(olympics_merge['NOC']=='TUV', 'Tuvalu', olympics_merge['Country'])


# Put these values from Country into Team
olympics_merge.drop('Team', axis = 1, inplace = True)
olympics_merge.rename(columns = {'Country': 'Team'}, inplace = True)

Checking again for mapping of NOC to team we find that each is mapped to a single value! 

# Merge GDP data
To effectively study factors that affect the medal tally of a country, we need to import the Country-Year wise GDP data. THe GDP data has Country name, code, year and the GDP value. We will remove some unnecessary columns [Indicator Name and Indicator Code].With the changes the GDP data is shown below-

In [ ]:
w_gdp = pd.read_csv('../input/country-wise-gdp-data/world_gdp.csv', skiprows = 3)

# Remove unnecessary columns
w_gdp.drop(['Indicator Name', 'Indicator Code'], axis = 1, inplace = True)

# The columns are the years for which the GDP has been recorded. This needs to brought into a single column for efficient
# merging.
w_gdp = pd.melt(w_gdp, id_vars = ['Country Name', 'Country Code'], var_name = 'Year', value_name = 'GDP')

# convert the year column to numeric
w_gdp['Year'] = pd.to_numeric(w_gdp['Year'])

w_gdp.head()

Before we actually merge, lets check if NOCs in the olympics data match with those in the Country Code.

In [ ]:
len(list(set(olympics_merge['NOC'].unique()) - set(w_gdp['Country Code'].unique())))

So, 108 NOCs in the olympics dataset dont have representation in the gdp data country codes. Is the name of the country a better way to merge?

In [ ]:
len(list(set(olympics_merge['Team'].unique()) - set(w_gdp['Country Name'].unique())))

So this looks promising! Only 6! Let's do what we can easily achieve, add a country code for each Team in the olympics dataset first and then merge using the Country Code. The data now has GDP data attached!

In [ ]:
# Merge to get country code
olympics_merge_ccode = olympics_merge.merge(w_gdp[['Country Name', 'Country Code']].drop_duplicates(),
                                            left_on = 'Team',
                                            right_on = 'Country Name',
                                            how = 'left')

olympics_merge_ccode.drop('Country Name', axis = 1, inplace = True)

# Merge to get gdp too
olympics_merge_gdp = olympics_merge_ccode.merge(w_gdp,
                                                left_on = ['Country Code', 'Year'],
                                                right_on = ['Country Code', 'Year'],
                                                how = 'left')

olympics_merge_gdp.drop('Country Name', axis = 1, inplace = True)

# Merge Population Data
Apart from GDP, population is also of import when looking at a countries performance. There are two trains of thought for this - <br>
1. Higher populations mean a bigger talent pool to choose from,
2. Its not GDP alone but GDP per capita that decides how much resources a country has.

In [ ]:
# Read in the population data
w_pop = pd.read_csv('../input/country-wise-population-data/world_pop.csv')

w_pop.drop(['Indicator Name', 'Indicator Code'], axis = 1, inplace = True)

w_pop = pd.melt(w_pop, id_vars = ['Country', 'Country Code'], var_name = 'Year', value_name = 'Population')

# Change the Year to integer type
w_pop['Year'] = pd.to_numeric(w_pop['Year'])

w_pop.head()

In [ ]:
olympics_complete = olympics_merge_gdp.merge(w_pop,
                                            left_on = ['Country Code', 'Year'],
                                            right_on= ['Country Code', 'Year'],
                                            how = 'left')

olympics_complete.drop('Country', axis = 1, inplace = True)

olympics_complete.head()

There are a lot of missing values in the result data - the main reasons are the attributes of the countries not found in the GDP and population masters and also the fact that Population and GDP are only for 1961 onwards while Olympics data is from 1896. Therefore, let's consider only data from 1961 onwards.

In [ ]:
olympics_complete.isnull().sum()

In [ ]:
# Lets take data from 1961 onwards only and for summer olympics only
olympics_complete_subset = olympics_complete.loc[(olympics_complete['Year'] > 1960) & (olympics_complete['Season'] == "Summer"), :]

# Reset row indices
olympics_complete_subset = olympics_complete_subset.reset_index()
olympics_complete_subset.head()

# Exploratory Data Analysis

## Who has the most medals across all editions of the olympics?
Medal tally is the sum of all medals won.

Let's create a column that captures whether or not a medal was won! It would be 1 if Medal column says Gold, Silver or Bronze and 0 otherwise.

In [ ]:
olympics_complete_subset['Medal_Won'] = np.where(olympics_complete_subset.loc[:,'Medal'] == 'DNW', 0, 1)


Before we find out the best olympics nations, we need to realise that each row of the data actually represents whether or not an athlete won a medal or not. This means that for team events, just a sum of the 'Medal_Won' column would give us an incorrect picture of the medal tally. 
​
We need to correct for this. First, lets analyse which events are team events. So, what events are team events?
#### If a team wins more than one gold medal for an event in an edition of the olympics, then that event is a team event.

In [ ]:
# Check whether number of medals won in a year for an event by a team exceeds 1. This indicates a team event.
identify_team_events = pd.pivot_table(olympics_complete_subset,
                                      index = ['Team', 'Year', 'Event'],
                                      columns = 'Medal',
                                      values = 'Medal_Won',
                                      aggfunc = 'sum',
                                     fill_value = 0).drop('DNW', axis = 1).reset_index()

identify_team_events = identify_team_events.loc[identify_team_events['Gold'] > 1, :]

team_sports = identify_team_events['Event'].unique()


The list of names of events where in a single edition multiple golds were given below. Although these are recognised as team events, they are not actually team events. Some examples include -
1. Gymnastics Women's Balance Beam 
2. Gymnastics Men's Horizontal Bar 
3. Swimming Women's 100 metres Freestyle 
4. Swimming Men's 50 metres Freestyle

Upon analysis, I found that these are actually single events but because two athletes had the same score/time, both were awarded the gold medal. I will remove these events from the list of team sports

In [ ]:
# Read in the Tokyo Medals 2021 dataset
medals_21 = pd.read_csv('../input/2021-olympics-medals-in-tokyo/Tokyo Medals 2021.csv')

medals_21.drop(['Rank By Total','Gold Medal','Silver Medal','Bronze Medal'] 'axis = 1, inplace = True)
medals_21['Year'] = pd.Series([2021 for x in range(len(medals_21.index))])
medals_21.head()


In [ ]:
olympics_complete_set = olympics_complete_subset.merge(medals_21,
                                            left_on = ['Team', 'Year'],
                                            right_on= ['Country', 'Year'],
                                            how = 'left')
display(olympics_complete_set)